# 03. Artificial Neural Networks with TF

<p style="text-align: right;">
rayleigh@dplus.company<br>
blackdew7@gmail.com<br>
Your name :
</p>

### Intro to TensorFlow 의 마지막 시간!<br>

이 실습자료까지 학습하고나면, 모두 끝납니다!

#### 선행지식
1. 모델링을 한다는 것에 대한 이해.
2. Logistic Regression, Linear Regression
3. Activation function 과 레이어

#### 실습목표
1. ANN 의 구조를 Graph로 그려낼 수 있다.
2. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.
3. ANN의 마지막 레이어에 따라서 Regression도 Classification도 가능함을 알고 코딩할 수 있다.

#### 사용데이터.

01. Multivariate Regression : http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset
02. Multinomial Classification : http://yann.lecun.com/exdb/mnist/

# Regression with Artificial Neural Network

실습 01의 Bike-Sharing Data를 기억하지요?

단순히 Linear Regression을 사용하기에는 이런 저런 고민들이 됩니다.

1. 변수가 저렇게 많으면 어차피 직관적인 해석은 안된다.
2. 저 중 컨트롤 가능한 변수는 없었다.
3. 어쨌든 인풋 데이터에 따라 아웃풋 데이터가 잘 나오면 된다.

바야흐로, Artificial Neural Network가 필요한 때입니다.

### Warm up before deep playing

### 00. 라이브러리 불러오기

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import os

# 현재 작업 디렉토리 확인
os.getcwd()

### 01. 저번에 했었던 모든 전처리들

In [ ]:
# 데이터 불러오기
data_path = './Bike-Sharing-Dataset/hour.csv'
rides = pd.read_csv(data_path)

# 더미
dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)

# 드롭
fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 'mnth', 'hr', 'weekday']
data = rides.drop(fields_to_drop, axis=1)

# 노말리이제이션(Scailing)
quant_features = ['casual', 'registered', 'cnt', 'temp', 'atemp', 'hum', 'windspeed']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean) / std
    
# 트레이닝 / 테스트 셋
test_data, train_data = data[-60*24:], data[:-60*24]

target_fields = ['cnt', 'casual', 'registered']
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]
train_features, train_targets = train_data.drop(target_fields, axis=1), train_data[target_fields]

test_X, test_Y = test_features.values, test_targets.values
train_X, train_Y = train_features.values, train_targets.values

print(train_X.shape, train_Y.shape)


### 02. 손으로 그려보자 ANN

일단은 같이 해봅시다!<br>
맨 마지막에, 아무것도 안주고 여러분이 해야만 하는 부분이 있습니다!

#### 조건 :
* Hidden Layer는 3개만 쓰자.
* Hidden Layer의 노드 수는 16, 8, 4 개로 한다.
* Shape를 실수하지 말자.

### 03. Make a Graph with TF

01.Linear Regression with TF 의 코드와 직접 비교해보셔도 좋습니다

In [ ]:
'''
print(train_X.shape, train_Y.shape)
(15435, 58),   (15435, 3)
'''
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01

# Input Layer, Real Y
X = tf.placeholder(dtype=tf.float32, shape=[None, 58], name="X")
Y = tf.placeholder(dtype=tf.float32, shape=[None, 3], name="Y")

# 1st Hidden Layer
W1 = tf.Variable(tf.random_normal([58, 16]), name='weight_1')
b1 = tf.Variable(tf.random_normal([16]), name='bias_1')
sum1 = tf.add(tf.matmul(X, W1), b1)
hidden_output1 = tf.nn.relu(sum1)

# 2nd Hidden Layer
W2 = tf.Variable(tf.random_normal([16, 8]), name='weight_2')
b2 = tf.Variable(tf.random_normal([8]), name='bias_2')
sum2 = tf.add(tf.matmul(hidden_output1, W2), b2)
hidden_output2 = tf.nn.relu(sum2)

# last Hidden Layer
W3 = tf.Variable(tf.random_normal([8, 8]), name='weight_3')
b3 = tf.Variable(tf.random_normal([8]), name = 'bias_3')
sum3 = tf.add(tf.matmul(hidden_output2, W3), b3)
hidden_output3 = tf.nn.relu(sum3)

# Output Layer
Wo = tf.Variable(tf.random_normal([8, 3]), name='weight_o')
bo = tf.Variable(tf.random_normal([3]), name='bias_o')
Y_pred = tf.add(tf.matmul(hidden_output3, Wo), bo, name='Y_predicted')

# Cost(loss) function & Optimizer
cost = tf.losses.mean_squared_error(Y, Y_pred)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


### 04. Train! Session!

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

start = time.clock()
for i in range(5000):
    # 학습 진행
    sess.run(optimizer, feed_dict={X: train_X, Y: train_Y})
    
    if (i % 100 == 0):
        # 학습 상황 디스플레이
        tr_l = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
        print("{}th training---- training loss: {:.5f}".format(i, tr_l))
        
print("Training Complete")
end = time.clock()

print("Total elapsed time: {}".format(end - start))
print("Test loss: {:.5f}".format(sess.run(cost, feed_dict={X: test_X, Y: test_Y})))

Y_predicted = sess.run(Y_pred, feed_dict={X: test_X, Y: test_Y})
sess.close()

### Check!

In [ ]:
# target_fields = ['cnt', 'casual', 'registered']
column = 'registered'
cindex = target_fields.index(column)
rows0, rows1 = 1000, 1100

fig, ax = plt.subplots(figsize=(10, 5))

# 예측값도 Scaling 되어 있으므로 그것을 원래대로 되돌려주는 과정
mean, std = scaled_features[column]
predictions = Y_predicted[rows0:rows1, cindex] * std + mean
REAL = test_Y[rows0:rows1, cindex] * std + mean

#ax.plot(Y_predicted, label = 'Prediction')
#ax.plot(test_Y, label = 'Real')
ax.plot(predictions, label='Prediction')
ax.plot(REAL, label='Real')
ax.set_xlim(right=len(Y_predicted[rows0:rows1, cindex]))
ax.legend()

dates = pd.to_datetime(rides.iloc[test_data[rows0:rows1].index]['dteday'])
dates = dates.apply(lambda d: d.strftime('%b %d'))
ax.set_xticks(np.arange(len(dates))[12::24])
_ = ax.set_xticklabels(dates[12::24], rotation=45)

In [ ]:
tf.reset_default_graph()
%reset

# Classification with Artificial Neural Network

실습 02의 MNIST Data를 기억하지요?

1. 변수가 저렇게 많으면 어차피 직관적인 해석은 안된다.
2. 저 중 컨트롤 가능한 변수는 없었다.
3. 어쨌든 인풋 데이터에 따라 아웃풋 데이터가 잘 나오면 된다.
4. 원리가 어쨌든 정확성이 높아야 한다.


### 외칩시다. NEURAL NETWORK!

### 00. 라이브러리 불러오기

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tensorflow.examples.tutorials.mnist import input_data

### 01. 저번에 했었던 모든 전처리들

In [ ]:
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

In [ ]:
mnist.test.labels.tolist()[0].index(1)

In [ ]:
'''
mnist 시각화 함수
'''
def mnist_plot(i):
    pixels = mnist.test.images[i].reshape((28, 28))
    a = mnist.test.labels.tolist()
    plt.title('Label : {}'.format(a[i].index(1)))
    plt.imshow(pixels, cmap='gray')
    return plt.show()

In [ ]:
'''
0 ~ 9999 사이의 숫자를 넣어서 확인해보자.
'''
mnist_plot(5)

In [ ]:
'''
트레이닝 데이터로 활용할 55000개 이미지, 테스트용 10000개 이미지
28*28 사이즈의 이미지가 그냥 주욱 784칸 짜리 어레이에 담겨있다.
레이블은 이미 one-hot encoding이 되어 있다.
'''
print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)

In [ ]:
mnist.train.labels[0]

### 02. 손으로 그려보자 ANN

일단은 같이 해봅시다!<br>
맨 마지막에, 아무것도 안주고 여러분이 해야만 하는 부분이 있습니다!

#### 조건 :
* Hidden Layer는 4개만 쓰자.
* Hidden Layer의 노드 수는 256, 128, 64, 64 개로 한다.
* Shape를 실수하지 말자.
* 인풋의 Shape = [ None, 28\*28 ], 아웃풋의 Shape = [ None, 10 ] 기억하자

### 03. Make a Graph with TF

02.Logistic Regression with TF 의 코드와 직접 비교해보셔도 좋습니다

### 02. 텐서서플로우로 그래프를 만들자!

In [ ]:
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01
training_epochs = 20
batch_size = 128

# Input Layer, Real Y
X = tf.placeholder(dtype=tf.float32, shape=[None, 784])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

# 1st Hidden Layer
W1 = tf.Variable(tf.random_normal([28 * 28, 512]), name='weight_1')
b1 = tf.Variable(tf.random_normal([512]), name='bias_1')
sum1 = tf.add(tf.matmul(X, W1), b1)
hidden_output1 = tf.nn.relu(sum1)

# 2nd Hidden Layer
W2 = tf.Variable(tf.random_normal([512, 512]))
b2 = tf.Variable(tf.random_normal([512]))
sum2 = tf.add(tf.matmul(hidden_output1, W2), b2)
hidden_output2 = tf.nn.relu(sum2)

# 3rd Hidden Layer
W3 = tf.Variable(tf.random_normal([512, 256]))
b3 = tf.Variable(tf.random_normal([256]))
sum3 = tf.add(tf.matmul(hidden_output2, W3), b3)
hidden_output3 = tf.nn.relu(sum3)

# 4nd Hidden Layer
W4 = tf.Variable(tf.random_normal([256, 256]))
b4 = tf.Variable(tf.random_normal([256]))
sum4 = tf.add(tf.matmul(hidden_output3, W4), b4)
hidden_output4 = tf.nn.relu(sum4)

# Output Layer
Wo = tf.Variable(tf.random_normal([256, 10]), name='weight_o')
bo = tf.Variable(tf.random_normal([10]), name='bias_o')
logits = tf.add(tf.matmul(hidden_output4, Wo), bo, name='logit')
Y_prob = tf.nn.softmax(logits, name='Y_distribution')

# Classification & Accuracy
n_of_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), 
                        name='Comparison')
accuracy = tf.reduce_mean(tf.cast(n_of_correct, tf.float32), 
                          name='accuracy')

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [ ]:
784 * 512 + 512 * 512 + 512 * 256 + 256 * 256 + 256 * 10

28 * 28

### 03. 세션을 만들고, 학습시키자!

In [ ]:
sess = tf.Session()
tf.set_random_seed(2017)
sess.run(tf.global_variables_initializer())

training_epochs = 20
batch_size = 128

for i in range(training_epochs + 1):
    # 배치의 개수 미리 준비
    n_of_batches = int(mnist.train.num_examples / batch_size)
    
    print("{}th epoch".format(i))
    for batch in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})
        
        if (batch % 25 == 0):
            tr_a, loss = sess.run([accuracy, cost], feed_dict={X: X_batch, Y: Y_batch})
            print("training loss: {:.4f} accuracy: {:.2f}%".format(loss, tr_a * 100))

print("Training Complete")
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
# print(sess.run([W[:, 0], b]))
# sess.close()

### 성능을 한 번 확인해 보자. 제대로 되고 있는걸까?

In [ ]:
n = int(np.random.uniform(0, 9999, 1))
n = 9719
Predicted_distribution = sess.run(Y_prob, feed_dict={X: [mnist.test.images[n]]})
Predicted_distribution = Predicted_distribution[0]
pd_dict = {i: '%.2f' % prob for i, prob in enumerate(Predicted_distribution)}

print(n)
print(pd_dict)
mnist_plot(n)


### 좀 더 멋지게
1. hyper parameter 중 - 변수 초기화 알고리즘 사용
2. 그래프를 보기에 더 좋게 하려면 => name_scope

In [ ]:
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01
training_epochs = 20
batch_size = 128

# Input Layer, Real Y
with tf.name_scope("Input_Layer"):
    X = tf.placeholder(dtype=tf.float32, shape=[None, 28 * 28], name='X_input')
with tf.name_scope("Labels"):
    Y = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='Y_input')

# 1st Hidden Layer
with tf.name_scope("Hidden_Layer_1"):
    W1 = tf.get_variable(name='Weight1', shape=[28 * 28, 512],
                         initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.truncated_normal([512], mean=0, stddev=0.5), name='Bias1')
    sum1 = tf.add(tf.matmul(X, W1), b1, name='sum1')
    hidden_output1 = tf.nn.relu(sum1 ,name='Output1')

# 2nd Hidden Layer
with tf.name_scope("Hidden_Layer_2"):
    W2 = tf.get_variable(name='Weight2', shape=[512, 512],
                         initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.truncated_normal([512], mean=0, stddev=0.5), name='Bias2')
    sum2 = tf.add(tf.matmul(hidden_output1, W2), b2, name='sum2')
    hidden_output2 = tf.nn.relu(sum2, name='Output2')

# 3rd Hidden Layer
with tf.name_scope("Hidden_Layer_3"):
    W3 = tf.get_variable(name='Weight3', shape=[512, 256],
                         initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.truncated_normal([256], mean=0, stddev=0.5), name='Bias3')
    sum3 = tf.add(tf.matmul(hidden_output2, W3), b3, name='sum3')
    hidden_output3 = tf.nn.relu(sum3, name='Output3')
    
# 4nd Hidden Layer
with tf.name_scope("Hidden_Layer_4"):
    W4 = tf.get_variable(name='Weight4', shape=[256, 128],
                         initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.truncated_normal([128], mean=0, stddev=0.5), name='Bias4')
    sum4 = tf.add(tf.matmul(hidden_output3, W4), b4, name='sum4')
    hidden_output4 = tf.nn.relu(sum4, name='Output4')

# Output Layer
with tf.name_scope("Output_Layer"):
    Wo = tf.get_variable(name='Weight_output', shape=[128, 10],
                         initializer=tf.contrib.layers.xavier_initializer())
    bo = tf.Variable(tf.truncated_normal([10], mean=0, stddev=0.5), name='Bias_output')
    logits = tf.add(tf.matmul(hidden_output4, Wo), bo, name='logit')
    Y_prob = tf.nn.softmax(logits, name='Y_distribution_predicted')

# Accuracy and cost
with tf.name_scope("Accuracy_and_Cost"):
    n_of_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), name='Comparison')
    accuracy = tf.reduce_mean(tf.cast(n_of_correct, tf.float32), name='Accuracy')
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name='Cross_entropy')

# Optimizer
with tf.name_scope("Optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [ ]:
sess = tf.Session()
tf.set_random_seed(2017)
sess.run(tf.global_variables_initializer())

training_epochs = 20
batch_size = 128

for i in range(training_epochs + 1):
    # 배치의 개수 미리 준비
    n_of_batches = int(fashion.train.num_examples / batch_size)
    
    print("{}th epoch".format(i))
    for batch in range(n_of_batches):
        X_batch, Y_batch = fashion.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})
        
        if (batch % 25 == 0):
            tr_a, loss = sess.run([accuracy, cost], feed_dict={X: X_batch, Y: Y_batch})
            print("training loss: {:.4f} accuracy: {:.2f}%".format(loss, tr_a * 100))
            
print("Training Complete")        
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
# print(sess.run([W[:, 0], b]))
sess.close()

In [ ]:
from tbij import show_graph
show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
tf.reset_default_graph()
%reset

# (CHALLENGE!) MNIST is too easy, huh?!

## Here Comes the 'FASHION MNIST'



### 00. 라이브러리 불러오기

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tensorflow.examples.tutorials.mnist import input_data
from tbij import show_graph

### 01.  모든 전처리들

In [ ]:
url = 'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/'
fashion = input_data.read_data_sets('./Fashion_data/',
                                    source_url=url, 
                                    one_hot=True)

In [ ]:
'''
Fashion mnist 시각화 함수
'''
def fashion_plot(i):
    pixels = fashion.test.images[i].reshape((28, 28))
    a = fashion.test.labels.tolist()
    plt.title('Label : {}'.format(a[i].index(1)))
    plt.imshow(pixels, cmap='gray')
    return plt.show()

| Label	| Description |
|:------|------:|
|0|T-shirt/top|
|1|	Trouser|
|2|	Pullover|
|3|	Dress|
|4|	Coat|
|5|	Sandal|
|6|	Shirt|
|7|	Sneaker|
|8|	Bag|
|9|	Ankle boot|

In [ ]:
'''
반복실행해보자
'''
fashion_plot(np.random.randint(0, high=9999))

In [ ]:
'''
트레이닝 데이터로 활용할 55000개 이미지, 테스트용 10000개 이미지
28*28 사이즈의 이미지가 그냥 주욱 784칸 짜리 어레이에 담겨있다.
레이블은 이미 one-hot encoding이 되어 있다.
'''
print(fashion.train.images.shape, fashion.train.labels.shape)
print(fashion.test.images.shape, fashion.test.labels.shape)

In [ ]:
######################################################
#                    코딩하시면 됩니다!
######################################################

# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01
training_epochs = 10
batch_size = 128

# Input Layer, Real Y
X = tf.placeholder(dtype=tf.float32, shape=[None, 784])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

# 1nd Hidden Layer
W1 = tf.Variable(tf.random_normal([784, 512]))
b1 = tf.Variable(tf.random_normal([512]))
sum1 = tf.add(tf.matmul(X, W1), b1)
hidden_output1 = tf.nn.relu(sum1)

# 2nd Hidden Layer
W2 = tf.Variable(tf.random_normal([512, 512]))
b2 = tf.Variable(tf.random_normal([512]))
sum2 = tf.add(tf.matmul(hidden_output1, W2), b2)
hidden_output2 = tf.nn.relu(sum2)

# 3rd Hidden Layer
W3 = tf.Variable(tf.random_normal([512, 256]))
b3 = tf.Variable(tf.random_normal([256]))
sum3 = tf.add(tf.matmul(hidden_output2, W3), b3)
hidden_output3 = tf.nn.relu(sum3)

# 4nd Hidden Layer
W4 = tf.Variable(tf.random_normal([256, 256]))
b4 = tf.Variable(tf.random_normal([256]))
sum4 = tf.add(tf.matmul(hidden_output3, W4), b4)
hidden_output4 = tf.nn.relu(sum4)

# Output Layer
Wo = tf.Variable(tf.random_normal([512, 10]))
bo = tf.Variable(tf.random_normal([10]))
logits = tf.add(tf.matmul(hidden_output2, Wo), bo)
Y_prob = tf.nn.softmax(logits, name='Y_distribution_predicted')

# Accuracy and cost
with tf.name_scope("Accuracy_and_Cost"):
    n_of_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), name='Comparison')
    accuracy = tf.reduce_mean(tf.cast(n_of_correct, tf.float32))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), 
                          name='Cross_entropy')

# Optimizer
with tf.name_scope("Optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [ ]:
sess = tf.Session()
tf.set_random_seed(2017)
sess.run(tf.global_variables_initializer())

training_epochs = 20
batch_size = 100

for i in range(training_epochs + 1):
    # 배치의 개수 미리 준비
    n_of_batches = int(fashion.train.num_examples / batch_size)
    
    print("{}th epoch".format(i))
    for batch in range(n_of_batches):
        X_batch, Y_batch = fashion.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})
        
        if (batch % 25 == 0):
            tr_a, loss = sess.run([accuracy, cost], feed_dict={X: X_batch, Y: Y_batch})
            print("training loss: {:.4f} accuracy: {:.2f}%".format(loss, tr_a * 100))

print("Training Complete")        
test_accuracy = sess.run(accuracy, feed_dict={X: fashion.test.images, Y: fashion.test.labels})
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
# print(sess.run([W[:, 0], b]))


In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
n = int(np.random.uniform(0, 9999, 1))
Predicted_distribution = sess.run(Y_prob, feed_dict={X: [fashion.test.images[n]]})
Predicted_distribution = Predicted_distribution[0]

labels_list = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankleboot']
pd_dict = {str(i) + ": " + labels_list[i]: '%.2f' % prob for i, prob in enumerate(Predicted_distribution)}

print(n)
print(pd_dict)
fashion_plot(n)
